In [1]:
! pip install -q openai pydantic

In [2]:
from enum import Enum
import json
import requests
from openai import OpenAI
from pydantic import BaseModel, Field

In [3]:
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')

In [4]:
client = OpenAI(api_key=openai_api_key)
MODEL = "gpt-4o-2024-08-06"

Structured Outputs via tools is available by setting **strict: True** within your function definition.

In [ ]:
response =  client.chat.completions.create(
    model = MODEL,
    messages = [
      {
        "role": "system",
        "content": "You are a helpful assistant. The current date is August 6, 2024. You help users query for the data they are looking for by calling the query function."
      },
      {
        "role": "user",
        "content": "look up all my orders in may of last year that were fulfilled but not delivered on time"
      }
    ],
    tools = [
      {
        "type": "function",
        "function": {
          "name": "query",
          "description": "Execute a query.",
          "strict": True,
          "parameters": {
            "type": "object",
            "properties": {
              "table_name": {
                "type": "string",
                "enum": ["orders"]
              },
              "columns": {
                "type": "array",
                "items": {
                  "type": "string",
                  "enum": [
                    "id",
                    "status",
                    "expected_delivery_date",
                    "delivered_at",
                    "shipped_at",
                    "ordered_at",
                    "canceled_at"
                  ]
                }
              },
              "conditions": {
                "type": "array",
                "items": {
                  "type": "object",
                  "properties": {
                    "column": {
                      "type": "string"
                    },
                    "operator": {
                      "type": "string",
                      "enum": ["=", ">", "<", ">=", "<=", "!="]
                    },
                    "value": {
                      "anyOf": [
                        {
                          "type": "string"
                        },
                        {
                          "type": "number"
                        },
                        {
                          "type": "object",
                          "properties": {
                            "column_name": {
                              "type": "string"
                            }
                          },
                          "required": ["column_name"],
                          "additionalProperties": False
                        }
                      ]
                    }
                  },
                  "required": ["column", "operator", "value"],
                  "additionalProperties": False
                }
              },
              "order_by": {
                "type": "string",
                "enum": ["asc", "desc"]
              }
            },
            "required": ["table_name", "columns", "conditions", "order_by"],
            "additionalProperties": False
          }
        }
      }
  ]
)

In [6]:
response

ChatCompletion(id='chatcmpl-AkuI0QsNOA4TtwTrRR0obxSW9O1cs', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_XH32AlGRFpsNRzgQB5t3Qk7h', function=Function(arguments='{"table_name":"orders","columns":["id","status","expected_delivery_date","delivered_at","shipped_at","ordered_at","canceled_at"],"conditions":[{"column":"ordered_at","operator":">=","value":"2023-05-01"},{"column":"ordered_at","operator":"<","value":"2023-06-01"},{"column":"status","operator":"=","value":"fulfilled"},{"column":"delivered_at","operator":">","value":{"column_name":"expected_delivery_date"}}],"order_by":"asc"}', name='query'), type='function')]))], created=1735742828, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_d28bcae782', usage=CompletionUsage(completion_tokens=118, prompt_tokens=186

In [19]:
answer = response.choices[0].message.tool_calls[0].function.arguments
answer

'{"table_name":"orders","columns":["id","status","expected_delivery_date","delivered_at","shipped_at","ordered_at","canceled_at"],"conditions":[{"column":"ordered_at","operator":">=","value":"2023-05-01"},{"column":"ordered_at","operator":"<","value":"2023-06-01"},{"column":"status","operator":"=","value":"fulfilled"},{"column":"delivered_at","operator":">","value":{"column_name":"expected_delivery_date"}}],"order_by":"asc"}'

In [21]:
parsed_answer = json.loads(answer)
parsed_answer

{'table_name': 'orders',
 'columns': ['id',
  'status',
  'expected_delivery_date',
  'delivered_at',
  'shipped_at',
  'ordered_at',
  'canceled_at'],
 'conditions': [{'column': 'ordered_at',
   'operator': '>=',
   'value': '2023-05-01'},
  {'column': 'ordered_at', 'operator': '<', 'value': '2023-06-01'},
  {'column': 'status', 'operator': '=', 'value': 'fulfilled'},
  {'column': 'delivered_at',
   'operator': '>',
   'value': {'column_name': 'expected_delivery_date'}}],
 'order_by': 'asc'}